## Belegaufgabe zum Blockkurs Python
# Simulation mit vtk
------

Ziel ist es ein Modul zu schreiben, welches eine Windmühle simuliert. Die Windmühle soll dabei nur
aus primitiven Objekten zusammengesetzt werden und einen Rotor besitzen, der sich während der
Simulation dreht. Durch Tastendruck auf + oder – soll sich die Rotationsgeschwindigkeit anpassen
lassen.

------
The aim is to write a module that simulates a windmill. The windmill should only be made up of primitive objects and have a rotor that rotates during the simulation. The rotation speed should be adjustable by pressing + or -.

------

# Todo
- [X] class for objects
- [X] assemle windmill
- [X] vtk ???
- [X] add dynamics
- ~~[ ] add buttons~~
- [X] add slider
- [X] add renders ps, rpm text



Global variables ```NUMBER_OF_WING```, ```REFRESH_RATE```, ```WINDOW_W```, ```WINDOW_H``` were used for hardcoded settings to be easily changed if needed.\
For the same reason ```material_dict``` dictionary was implemented, just to set colors and opacities for each material in one place alltogether


In [67]:
import numpy as np
import vtk

NUMBER_OF_WING = 3
REFRESH_RATE = 100 # Hz
WINDOW_W, WINDOW_H = 800, 600

material_dict = {}

def add_material(name: str, material_properties: tuple):
    if name not in material_dict:
        material_dict[name] = material_properties
    else: 
        raise Exception("Such material already exists")
    
def get_material_list():
    return material_dict.keys()


Rotation is implemented through regular rotation matrix that is getting multiplied by orientation matrix of the body.\
At the same time, position vector after rotation = rotation matrix @ initial position

In [68]:
class Structure:
    #main class for all the structures
    def __init__(self, material):
        self.material = material

        self.vtk_body = None

        #self.position = np.zeros(3)
        self.position = np.array([0.,0.,0.])
        self.orientation = np.eye(3)
        
        self.actor = vtk.vtkActor()

    # move center of the body by the vector (x,y,z)
    def move_by(self, vect):
        self.position += vect
        self.update()

    # rotate the body around the axis origin, for example around Oz with body center moving around too
    def rotate_by(self, theta, axis='z'):
        if axis == 'z':
            rotation_matrix = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
        elif axis == 'y':
            rotation_matrix = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                                        [0, 1, 0],
                                        [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'x':
            rotation_matrix = np.matrix([[1, 0, 0],
                                        [0, np.cos(theta), -np.sin(theta)],
                                        [0, np.sin(theta), np.cos(theta)]])
        else:
            raise Exception("Axis should be either \'x\', \'y\' or \'z\'")
        
        pos_vector = self.position.reshape(3,1)  # make it vertical vector
        pos_vector = rotation_matrix @ pos_vector
        self.position = pos_vector.reshape(1,3) # make it horizontal again

        self.orientation = self.orientation * rotation_matrix

        self.update()

    # spin the body around the axis parallel to z(or x or y) and goes throug the center of the body (was not needed tho)    
    def spin_by(self, theta, axis='z'):
        if axis == 'z':
            rotation_matrix = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                                        [np.sin(theta), np.cos(theta), 0],
                                        [0, 0, 1]])
        elif axis == 'y':
            rotation_matrix = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                                        [0, 1, 0],
                                        [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'x':
            rotation_matrix = np.matrix([[1, 0, 0],
                                        [0, np.cos(theta), -np.sin(theta)],
                                        [0, np.sin(theta), np.cos(theta)]])
        else:
            raise Exception("Axis should be either \'x\', \'y\' or \'z\'")
        
        self.orientation = self.orientation * rotation_matrix

        self.update()

    # initiate mapper for the body
    def create_mapper(self):
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(self.vtk_body.GetOutputPort())
        
        self.actor.SetMapper(mapper)

    # visualization tweaking
    def set_edge_visibility(self, on=False):
        if on:
            self.actor.GetProperty().EdgeVisibilityOn()
            self.actor.GetProperty().SetLineWidth(2)
        else:
            self.actor.GetProperty().EdgeVisibilityOff()

    def set_color(self, color: tuple[float]):
        self.actor.GetProperty().SetColor(color)
        
    def set_opacity(self, opacity):
        self.actor.GetProperty().SetOpacity(opacity)

    # make a pokematrix for the actor
    def update(self):
        pokematrix = np.eye(4,4)
        pokematrix[:,-1][:3] = self.position
        pokematrix[:3][...,:3] = self.orientation
        vtk_poke = vtk.vtkMatrix4x4()
        for i in range(4):
            for j in range(4):
                vtk_poke.SetElement(i,j, pokematrix[i,j])
        self.actor.PokeMatrix(vtk_poke)

    # structure position
    @property
    def get_position(self):
        return self.position

    # structure orientation
    @property
    def get_orientation(self):
        return self.orientation



Main bodies inherit all the methods of the ```Structure class``` + get
- dimensions parameters (simply ```L x W x H``` or ```R x H``` for cylinder)
- material parameter (for the visualization reasons)
- obtain ```vtk_body``` of required shape (Cube, Cylinder, Cone)
- ```Protector class``` also gets resolution by default set to ```NUMBER_OF_WING*2``` because it looks good

In [69]:
# Just represents ground
class Fundament(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(l)
        self.vtk_body.SetYLength(h)
        self.vtk_body.SetZLength(w)
        self.dimensions = (l,h,w)

        self.create_mapper()

# Represents pillar that holds rotor and fan
class Tube(Structure):
    def __init__(self, material, r, h, res=20):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCylinderSource()
        
        self.vtk_body.SetRadius(r)
        self.vtk_body.SetHeight(h)
        self.vtk_body.SetResolution(res)
        self.dimensions = (r,h,r)
        
        self.create_mapper()

# Represents rotor to which fan is adjusted
class Rotor(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(l)
        self.vtk_body.SetYLength(h)
        self.vtk_body.SetZLength(w)
        self.dimensions = (l,h,w)

        self.create_mapper()

# Represents one wing of a fan, is used to assemble whole fan
class Wing(Structure):
    def __init__(self, material, l, w, h):
        super().__init__(material)
        
        self.vtk_body = vtk.vtkCubeSource()
        
        self.vtk_body.SetXLength(h)
        self.vtk_body.SetYLength(l)
        self.vtk_body.SetZLength(w)

        self.dimensions = (h,l,w)

        self.create_mapper()

# Represents protection cone on the shaft of the fan
class Protector(Structure):
    def __init__(self, material, r, h, res=NUMBER_OF_WING*2):
        super().__init__(material)
        self.vtk_body = vtk.vtkConeSource()
        
        self.vtk_body.SetRadius(r)
        self.vtk_body.SetHeight(h)
        self.vtk_body.SetResolution(res)
        self.dimensions = (r,h,r)
        
        self.create_mapper()


Class ```Fan``` is assembling one vtk actor from the dynamically created ```NUMBER_OF_WING``` ```Wing objects``` (that are stored in the list ```winglist```)\
For further usage of this assembled actor, the ```Fan``` object also has dimensions, position and orientation parameters. ```set_color()```, ```set_opacity()```, and ```set_edge_visibility()``` methodes had to be reintroduce to work properly.\
The inherited ```move_by()``` and ```rotate_by()``` methods work perfectly fine with newly introduced position and orientation.

In [70]:

class Fan(Structure):
    global NUMBER_OF_WING
    def __init__(self, winglist: list[Wing]):
        self.material = winglist[0].material
        self.winglist = winglist
        self.assembly = vtk.vtkAssembly()
        self.dimensions = (winglist[0].dimensions[0], winglist[0].dimensions[1] * 2, winglist[0].dimensions[2])
        self.position = np.array([0.,0.,0.])
        self.orientation = np.eye(3)

        for i in range(NUMBER_OF_WING):
          
            self.winglist[i].move_by((0, self.winglist[0].dimensions[1] * 0.5, 0))
            self.winglist[i].rotate_by(i*2*np.pi/NUMBER_OF_WING)
            self.assembly.AddPart(self.winglist[i].actor)

        self.actor = self.assembly
    
    def set_color(self, color: tuple[float]):
        for winginstance in self.winglist:
            winginstance.set_color(color)
    
    def set_opacity(self, opacity):
        for winginstance in self.winglist:
            winginstance.set_opacity(opacity)
    
    def set_edge_visibility(self, on=False):
        for winginstance in self.winglist:
            winginstance.set_edge_visibility(on)


Class ```Windmill``` is the core class of this project, where all the magic happens.\
\
It does not inherit any other classes, it takes 5 objects as args
- ```fund```, ```tube```, ```rotor```, ```fan```, ```prot``` are self-explaining,
- `self.part` is needed to iterate through all the objects
- ```tick``` is the length of each refresh in ms, in this case hardcoded from ```REFRESH_RATE``` global parameter, ```rps``` is variable to store renders per second value\
---
- `arrange_parts()` moves and rotates all objects to take correct position with respect to the dimensions of the objects (for this reason all of the structure subclass objects have `dimensions` list parameter)
- `paint_parts()` sets colors and opacities of each part according to the `material_dict`

In [71]:
  
class Windmill():
    def __init__(self, fund: Fundament, tube: Tube, rotor: Rotor, fan: Fan, prot: Protector):
        # initial objects that make a Windmill
        self.fund = fund
        self.tube = tube
        self.rotor = rotor
        self.fan = fan
        self.prot = prot
        
        self.parts = [self.fund, self.tube, self.rotor, self.fan, self.prot]
        
        # refresh window each tick, tick lasts for self.tick ms
        self.tick = int(1000/REFRESH_RATE)
        # default rpm
        self.rpm = 20
        # renders per second
        self.rps = 0.0

        self.jokemode = False


    def arrange_parts(self):
        # fan is located in (0,0,0) position
        # rotor is moved by 1/2 rotor length and 1/2 of fan length in z direction
        self.rotor.move_by((0, 
                           0,
                           -self.rotor.dimensions[2] * 0.5 - self.fan.dimensions[2] * 0.5))
        # tube has the same offset in z direction and also 1/2 tube H in y direction
        self.tube.move_by((0,
                          -self.tube.dimensions[1] * 0.5 - self.rotor.dimensions[1] * 0.5,
                          -self.rotor.dimensions[2] * 0.5- self.fan.dimensions[2] * 0.5))
        # fundament z offset is the same, y offset is H of the tube + 1/2 fundament dimension
        self.fund.move_by((0,
                          -self.tube.dimensions[1] - self.rotor.dimensions[1] * 0.5 - self.fund.dimensions[1] * 0.5,
                          -self.rotor.dimensions[2] * 0.5))
        # protector has to be rotated to point in z direction first
        self.prot.rotate_by(3* np.pi / 2, 'y')
        # and offset in z direction is 1/2 fan thickness + 1/2 protector H
        self.prot.move_by((0,
                           0,
                           self.prot.dimensions[1] * 0.5 + self.fan.dimensions[2] * 0.5))
        
    # this is optimized code above with the parts list
    def paint_parts(self):
        for part in self.parts:
            part.set_color(tuple(x/255 for x in material_dict[part.material]['Color']))
            part.set_opacity(material_dict[part.material]['Opacity'])  
            part.set_edge_visibility(True)
    
    # method to rotate whatever needs rotation
    def spin_all(self, angle):
        if self.jokemode:
            self.fund.rotate_by(angle, 'z')
            self.tube.rotate_by(angle, 'z')
            self.rotor.rotate_by(angle, 'z')
        else:
            self.fan.spin_by(angle, 'z')

    # callable method for slider interaction to change rpm
    def change_speed(self, *args):
        # print to console for debugging reasons
        print("interaction called, new speed = %d" % (self.slider.GetRepresentation().GetValue()))
        self.rpm = self.slider.GetRepresentation().GetValue()

    # callable method to update renders per second value
    def rps_counter(self, *args):
        rendertime =  self.rendr.GetLastRenderTimeInSeconds()
        self.rps = 1.0 / rendertime
        
   # method that creates renderer, render window, 
    def render_parts(self):
        # renderer and renderWindow setup
        self.rendr = vtk.vtkRenderer()
        self.rendrwind = vtk.vtkRenderWindow()
        self.rendrwind.AddRenderer(self.rendr)
        self.rendr.SetBackground(150/255, 100/255, 140/255)
        self.rendrwind.SetSize(WINDOW_W, WINDOW_H)
        self.rendrwind.SetWindowName('Windmill Simulation') # does nothing idk why

        # adding actors to the renderer
        for part in self.parts:
            self.rendr.AddActor(part.actor)

        # text actor to be able to print text to window
        self.textactor = self.show_text()
        self.rendr.AddActor(self.textactor)

        #interactor set up
        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.rendrwind)
        self.interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
        self.interactor.Initialize()
        
        # after initialization of the interactor we can add widget for slider
        self.slider = self.add_slider()

        # as well as observers:
        # TimerEvent interactor observer updates scene each tick ms
        self.interactor.AddObserver('TimerEvent', self.update_scene)
        self.interactor.CreateRepeatingTimer(self.tick)
        # EndInteractionEvent slider observer updates current rpm as soon as slider stops on new value
        self.slider.AddObserver("EndInteractionEvent", self.change_speed)
        # EndEvent renderer observer counts rps each time render is ended
        self.rendr.AddObserver('EndEvent', self.rps_counter)


    # method that creates text actor
    def show_text(self, fontsize = 18, color = (1,1,1)):
        txt = vtk.vtkTextActor()
        txt.SetPosition(8, 6)
        txt.GetTextProperty().SetFontSize(fontsize)
        txt.GetTextProperty().SetJustificationToLeft()
        txt.GetTextProperty().SetVerticalJustificationToBottom()
        txt.GetTextProperty().BoldOn()
        txt.GetTextProperty().SetColor(*color)
        txt.GetTextProperty().SetFontFamilyToArial()
        
        return txt

    # method that creates slider widget
    def add_slider(self):
        #setting up representation object for the slider
        slider_repr = vtk.vtkSliderRepresentation2D()
        
        slider_repr.SetMinimumValue(0.0)
        slider_repr.SetMaximumValue(1000.0)
        # initial value = self.rpm
        slider_repr.SetValue(self.rpm)
        slider_repr.SetTitleText("rpm")
                
        slider_repr.GetSliderProperty().SetColor(0.18, 0.77, 0.70)
        slider_repr.GetTitleProperty().SetColor(0.55, 0.90, 0.86)
        slider_repr.GetLabelProperty().SetColor(0.07, 0.49, 0.44)
        slider_repr.GetSelectedProperty().SetColor(0.70, 0.94, 0.91)
        slider_repr.GetTubeProperty().SetColor(0.16, 0.61, 0.56)
        slider_repr.GetCapProperty().SetColor(0.07, 0.49, 0.44)
        slider_repr.SetSliderLength(.04)
        slider_repr.SetSliderWidth(.04)
        slider_repr.SetEndCapLength(.015)

        # vertical slider on the right side
        slider_repr.GetPoint1Coordinate().SetCoordinateSystemToNormalizedDisplay()
        slider_repr.GetPoint1Coordinate().SetValue(0.9, 0.9)
        slider_repr.GetPoint2Coordinate().SetCoordinateSystemToNormalizedDisplay()
        slider_repr.GetPoint2Coordinate().SetValue(0.9, 0.1)

        slider_repr.SetLabelFormat("%.1f")

        # creating widget for the slider with chosen parameters
        widget = vtk.vtkSliderWidget()
        widget.SetInteractor(self.interactor)
        widget.SetRepresentation(slider_repr)

        # if the cap is clicked the slider will move to the cap
        widget.SetAnimationModeToAnimate()
        widget.SetNumberOfAnimationSteps(200)

        # set widget on
        widget.EnabledOn() 
        return widget

    # method that re-renders the scene, rotating the reqired actors by the angle
    def update_scene(self, *args):
        # i suppose i calculated this correctly
        angle =  2* np.pi * self.tick / 1000 * self.rpm / 60
    
        self.spin_all(angle)

        # refresh text on the rendered window
        self.textactor.SetInput('Current speed: %.2f rpm \nCurrent renders/sec: %.2f' % (self.rpm, self.rps))
        # re-render window
        self.interactor.GetRenderWindow().Render()

    # method to do everything in one place
    def visualize(self):
        self.arrange_parts()
        self.paint_parts()
        self.render_parts()
        
        self.interactor.Start()
        self.interactor.GetRenderWindow().Finalize()



And the main code that does everything

In [74]:
if __name__ == "__main__":
    # adding materials to the dictionary
    add_material("Steel", {"Color": (145, 161, 163), "Opacity": 1.0})  
    add_material("Wood", {"Color": (128, 56, 1), "Opacity": 0.95})
    add_material("Glass", {"Color": (209, 243, 255), "Opacity": 0.9})
    add_material("Concrete", {"Color": (124, 135, 135), "Opacity": 1.0})
    add_material("Bronze", {"Color": (190,93,4), "Opacity": 1.0})
    add_material("Copper", {"Color": (25, 120, 85), "Opacity": 1.0})

    # creating objects with (material, dimensions)
    Tube1 = Tube("Glass", 4, 120)
    Fundament1 = Fundament("Bronze", 100, 100, 12)
    Rotor1 = Rotor("Steel", 18, 18, 16)
    Protector1 = Protector("Copper", 4, 6)

    # dynamically creating NUMBER_OF_WING instances of Wing class 
    wings_list = [Wing("Wood", 60, 2, 6) for _ in range(NUMBER_OF_WING)]

    # creating a Fan object from these Wings
    Fan1 = Fan(wings_list)
    
    # assembling the Windmill from the parts
    Windmill1 = Windmill(Fundament1, Tube1, Rotor1, Fan1, Protector1)
    
    # visualizing the Windmill 
    Windmill1.visualize()

    # because of Jupyter cells we need to clear the dict so we can re-run just the last cell again:
    material_dict = {}



Calculator for the color values from the palette:

In [73]:
r,g,b = 178, 240, 232
print("%.2f, %.2f, %.2f" % (r/255, g/255, b/255))

#color 1 0.07, 0.49, 0.44
#color 2 0.16, 0.61, 0.56
#color 3 0.18, 0.77, 0.70
#color 4 0.55, 0.90, 0.86
#color 5 0.70, 0.94, 0.91


0.70, 0.94, 0.91
